importing the dependencies

In [9]:
import numpy as np
import pandas as pd
import re # re stands for "regular expression"
from nltk.corpus import stopwords #nltk means "natural language tool kit"
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vamsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
#printing the stopwords in english, doesn't add much value to the data during stemming process
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data pre-processing

In [12]:
# loading the data into pandas DataFrame
news_data = pd.read_csv("C:\\Users\\vamsi\\OneDrive\\Documents\\my project\\train.csv")

In [13]:
#no.of rows and colums
news_data.shape

(20800, 5)

In [14]:
#printing first 5 rows of the data frame
news_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
#counting the no.of missing values in the data set
news_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [16]:
#replacing the null values with empty string
news_data = news_data.fillna('')

In [17]:
# merging the author name and title
news_data['content'] = news_data['author']+news_data['title']
news_data

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...
...,...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0,Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0,"Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat..."
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0,Michael J. de la Merced and Rachel AbramsMacy’...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1,"Alex AnsaryNATO, Russia To Hold Parallel Exerc..."


In [18]:
#seperating the data and labels
X = news_data.drop('label',axis=1)
Y = news_data['label']

In [19]:
port_stem = PorterStemmer()

In [20]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
news_data['content'] = news_data['content'].apply(stemming)

In [ ]:
print(news_data['content'].head())

In [ ]:
# saperating the data and labels
X=news_data['content'].values
Y=news_data['label'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
#converting the text(news_data['content']) into numerical data
vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7)
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
print(X)

splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train , Y_test = train_test_split(X, Y, test_size =0.2, stratify=Y, random_state=2)

Training the model : Logistic regression 

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

evaluating the accuracy

In [ ]:
#accuracy score on the training data 
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [ ]:
print("accuracy score of the training data: ", training_data_accuracy)

In [ ]:
#accuracy score on the test data 
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print("accuracy score of the test data: ", test_data_accuracy)

making a predictive system

In [ ]:
x_new = X_test[]
prediction = model.predict(x_new)
print(prediction)
if (prediction[0]==0):
    print("news is real")
else:
    print("the news is fake")

In [ ]:
print(Y_test[0])

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
import pickle

# Create and train the model
pac = PassiveAggressiveClassifier(max_iter=1000)
pac.fit(X_train, Y_train)  # Assuming X_train and y_train are defined

# Save the trained model
filename = 'finalized_model.pkl'
pickle.dump(pac, open(filename, 'wb'))

In [ ]:
filename='vectorizer.pkl'
pickle.dump(vectorizer, open(filename, 'wb'))